In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
path_data = "../data"

In [3]:
filename_ligne = "Ligne.csv"

In [4]:
path_data_ligne = os.path.join(path_data, filename_ligne)

In [5]:
df_ligne = pd.read_csv(path_data_ligne, sep=",")


c:\users\ahmed\appdata\roaming\jetbrains\dataspell2021.3\projects\workspace\venv\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df_ligne

,N° CDE,L,Qte,Type,CodeClient,DateCreation,Gestionnaire,Type Menuiserie,Ouverture,Nb Zone,CA HT,Transformation,Unnamed: 12
0,140500651,1.0,1.0,C,2625.0,4/10/2018,ALU,HO,OF,1,1032.90,140500651-02,NaN
1,140500651,2.0,1.0,C,2625.0,4/10/2018,ALU,HO,OF,1,1032.90,140500651-02,NaN
2,140600145,1.0,8.0,C,5505.0,4/18/2018,ALU,FE,OF,1,13566.08,140600145-03,NaN
3,140600145,2.0,1.0,C,5505.0,4/18/2018,ALU,PF,OF,1,2126.06,140600145-03,NaN
4,140600145,3.0,1.0,C,5505.0,4/18/2018,ALU,FE,OF,1,1268.37,140600145-03,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133753,220400553-00,1.0,1.0,D,7702.0,4/13/2022,BOIS,ODB,BAS,1,608.00,NaN,NaN
133754,220400560-00,1.0,1.0,D,5959.0,4/13/2022,ALU,ODB,FX,1,1138.45,NaN,NaN
133755,220400560-00,2.0,0.0,D,5959.0,4/13/2022,ALU,ODB,FX,2,0.00,NaN,NaN
133756,220400561-00,1.0,1.0,D,12560.0,4/13/2022,ALU,FE,OF,1,1374.98,NaN,NaN


In [7]:
dfLigneTypeMenuiseriRepliable = df_ligne.loc[
    (df_ligne["Type Menuiserie"] == "MENREPLI") & (df_ligne["CA HT"] > 0)
    ]

print(dfLigneTypeMenuiseriRepliable)

              N° CDE     L  Qte Type  CodeClient DateCreation Gestionnaire  \
28690   190201456-04  32.0  1.0    D      1788.0    1/17/2020          ALU   
28704   190201456-05   4.0  1.0    D      1788.0    2/24/2020          ALU   
37572      190600147   1.0  1.0    C     10510.0    2/12/2020          ALU   
37573      190600147   2.0  1.0    C     10510.0    2/12/2020          ALU   
37575   190600147-01   2.0  2.0    D     10510.0    11/4/2019          ALU   
...              ...   ...  ...  ...         ...          ...          ...   
133538  220400332-00   1.0  1.0    D      4999.0     4/8/2022          ALU   
133570  220400351-00   1.0  1.0    D       915.0     4/8/2022          ALU   
133616  220400395-00   1.0  1.0    D       810.0     4/8/2022          ALU   
133657  220400434-00   1.0  1.0    D     11117.0    4/11/2022          ALU   
133699  220400473-00   1.0  1.0    D      4294.0    4/12/2022          ALU   

       Type Menuiserie Ouverture Nb Zone    CA HT Transformatio

In [8]:
dfLigneTypeMenuiserieCodeClient = dfLigneTypeMenuiseriRepliable.loc[:,
                                  ["Type", "CodeClient", "Type Menuiserie", "DateCreation"]]

dfLigneTypeMenuiserieCodeClient = dfLigneTypeMenuiserieCodeClient.rename(columns={'DateCreation': 'Nombre'})

print(dfLigneTypeMenuiserieCodeClient.groupby(["CodeClient", "Type"])["Nombre"].count().reset_index())

     CodeClient Type  Nombre
0          33.0    C       2
1          33.0    D       8
2          43.0    D       3
3          45.0    D       2
4          63.0    D       2
..          ...  ...     ...
413     14641.0    D       1
414     14649.0    D       1
415     14663.0    D       1
416     14691.0    C       1
417     14691.0    D       2

[418 rows x 3 columns]


In [9]:
dfLigneTypeMenuiserieCodeClientTransfo = pd.pivot_table(
    dfLigneTypeMenuiserieCodeClient,
    index=['CodeClient'],
    columns='Type',
    values='Nombre',
    aggfunc='count',
).reset_index().rename_axis(None, axis=1)

dfLigneTypeMenuiserieCodeClientTransfo['D'] = dfLigneTypeMenuiserieCodeClientTransfo['D'].fillna(0)
dfLigneTypeMenuiserieCodeClientTransfo['C'] = dfLigneTypeMenuiserieCodeClientTransfo['C'].fillna(0)

dfLigneTypeMenuiserieCodeClientTransfo = dfLigneTypeMenuiserieCodeClientTransfo.astype({
    "CodeClient": int,
    'D': int,
    'C': int
})

dfLigneTypeMenuiserieCodeClientTransfo

,CodeClient,C,D
0,33,2,8
1,43,0,3
2,45,0,2
3,63,0,2
4,113,0,1
...,...,...,...
371,14603,0,3
372,14641,0,1
373,14649,0,1
374,14663,0,1


In [11]:
dfLigneTypeMenuiserieCodeClientTransfo["TauxDeTransformation"] = dfLigneTypeMenuiserieCodeClientTransfo["C"] /dfLigneTypeMenuiserieCodeClientTransfo["D"] * 100

In [12]:
dfLigneTypeMenuiserieCodeClientTransfo.replace([np.inf, -np.inf], 1, inplace=True)

In [13]:
dfLigneTypeMenuiserieCodeClientTransfo

,CodeClient,C,D,TauxDeTransformation
0,33,2,8,25.0
1,43,0,3,0.0
2,45,0,2,0.0
3,63,0,2,0.0
4,113,0,1,0.0
...,...,...,...,...
371,14603,0,3,0.0
372,14641,0,1,0.0
373,14649,0,1,0.0
374,14663,0,1,0.0


In [32]:
dfLigneTypeMenuiserieCodeClientTransfo = dfLigneTypeMenuiserieCodeClientTransfo[
    (dfLigneTypeMenuiserieCodeClientTransfo["TauxDeTransformation"] >= 30) &
    (dfLigneTypeMenuiserieCodeClientTransfo["TauxDeTransformation"] < 40)
    ]

In [33]:
dfLigneTypeMenuiserieCodeClientTransfo

,CodeClient,C,D,TauxDeTransformation
